In [87]:
from bs4 import BeautifulSoup
import requests

#input user parameters eventually

#get results listing of therapists in BK
url = 'https://www.psychologytoday.com/us/therapists/ny/brooklyn'
# header added after running requests.get(url) by itself returned a 403 status code
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}
mainpagehtml = requests.get(url,headers=headers)

In [88]:
#check to make sure page connection is good
mainpagehtml.status_code

200

In [261]:
# turn html into beautifulsoup object and print so I can look at how the page is structured
soup = BeautifulSoup(mainpagehtml.content,'html.parser')
mainpage = soup.body
print(mainpage.prettify())

<body>
 <!-- Start Google Tag Manager -->
 <noscript>
  <iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-WGTKPG" style="display:none;visibility:hidden" width="0">
  </iframe>
 </noscript>
 <script>
  (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-WGTKPG');


    $(document).ready(function () {
        $('#enabled-mobile-nav-control').click(function (event) {
            event.stopPropagation();
            event.preventDefault();
            $('body').addClass('left-sidebar-engaged');
        });


        $('.has-country-submenu').click(function (event) {
            $('body').addClass('country-sidebar-engaged');
        });

        $('.sidebar-back-arrow

In [270]:
# filter to just get therapist results. without [0] t is ResultSet, with [0] it's a Tag
t = soup.find_all('div',{'class': 'result-row normal-result row'})[0]
t

<div class="result-row normal-result row" data-email-url="https://www.psychologytoday.com/us/profile/102234/contact" data-new-clients="1" data-phone="(646) 755-9294" data-prof-name="Amira R Crawford" data-profid="102234" data-profile-image="https://cdn0.sussexdirectories.com/rms/rms_photos/sized/34/22/102234_1_120x150.jpg?pu=0" data-profile-url="https://www.psychologytoday.com/us/therapists/amira-r-crawford-brooklyn-ny/102234" data-result-row="1" data-result-url="https://www.psychologytoday.com/us/therapists/ny/brooklyn/102234?sid=5d9f61b0a3a95&amp;ref=1" data-use-company-name="" itemscope="" itemtype="http://schema.org/Person">
<div class="col-4 col-sm-4 col-md-2 col-lg-2 photo-column">
<meta content="https://cdn0.sussexdirectories.com/rms/rms_photos/sized/34/22/102234_1_120x150.jpg?pu=0" itemprop="image">
<a href="https://www.psychologytoday.com/us/therapists/ny/brooklyn/102234?sid=5d9f61b0a3a95&amp;ref=1&amp;tr=ResultsPhoto">
<img alt="Amira R Crawford, Clinical Social Work/Therapis

In [139]:
# span is the tag that contains name, job title, address, etc. 
# Using this for now (eventually will pull profile link and get info from there; this is to start)

# get everything with a span tag
tspans = t.find_all('span')

# get list of the properties for each span tag
spanitemprops = []
for line in tspans:
    # if there's an 'itemprop' property, add it to the list spanitemprops
    if line.get('itemprop'): spanitemprops.append(line.get('itemprop'))

print('SPAN ITEMPROP LIST: \n', spanitemprops)

SPAN ITEMPROP LIST: 
 ['name', 'jobTitle', 'addressLocality', 'addressRegion', 'postalCode', 'workLocation', 'geo', 'addressLocality', 'addressRegion', 'postalCode']


DIV ITEMPROP LIST: 
 ['address', 'description', 'address', 'description']


In [342]:
infodict = {}

# list just unique properties
spanitemprops_u = set(spanitemprops)

# search through t and grab things with those itemprop values ; this can take a list of strings and search for 
# itemprops that match any string in the list. This produces a list of tags with those matching itemprops
propertieslist = t('span',{'itemprop':spanitemprops_u})
propertieslist

[<span itemprop="name">Amira R Crawford</span>,
 <span itemprop="jobTitle">Clinical Social Work/Therapist</span>,
 <span itemprop="addressLocality">Brooklyn,</span>,
 <span itemprop="addressRegion">New York</span>,
 <span itemprop="postalCode">11233</span>,
 <span itemprop="workLocation" itemscope="" itemtype="http://schema.org/Place">
 <span itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates">
 <meta content="40.683011" itemprop="latitude"/>
 <meta content="-73.9376937" itemprop="longitude"/>
 </span>
 </span>,
 <span itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates">
 <meta content="40.683011" itemprop="latitude"/>
 <meta content="-73.9376937" itemprop="longitude"/>
 </span>,
 <span itemprop="addressLocality">Brooklyn,</span>,
 <span itemprop="addressRegion">New York</span>,
 <span itemprop="postalCode">11233</span>]

In [343]:
# go through list of matching tags. for each tag, add the itemprop value as dict key and tag's string as dict value
for each in propertieslist:
    if each.get_text():
        infodict[each.get('itemprop')] = each.get_text()

# what is the best way to remove the '\n...' entries?
infodict

{'name': 'Amira R Crawford',
 'jobTitle': 'Clinical Social Work/Therapist',
 'addressLocality': 'Brooklyn,',
 'addressRegion': 'New York',
 'postalCode': '11233',
 'workLocation': '\n\n\n\n\n',
 'geo': '\n\n\n'}

FROM HERE, GETTING INFO FROM THERAPIST'S PROFILE PAGE

In [313]:
tempurl = t.find('a',{'class':'result-name'}).get('href')
if tempurl:
    tempurl = BeautifulSoup(requests.get(tempurl,headers=headers).content,'html.parser').body
    
print(tempurl.prettify())

<body class="">
 <!-- Start Google Tag Manager -->
 <noscript>
  <iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-WGTKPG" style="display:none;visibility:hidden" width="0">
  </iframe>
 </noscript>
 <script>
  (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-WGTKPG');


    $(document).ready(function () {
        $('#enabled-mobile-nav-control').click(function (event) {
            event.stopPropagation();
            event.preventDefault();
            $('body').addClass('left-sidebar-engaged');
        });


        $('.has-country-submenu').click(function (event) {
            $('body').addClass('country-sidebar-engaged');
        });

        $('.sidebar-b

In [316]:
for each in tempurl:
    try:
        if each.children:
            print(each)
    except: pass

<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-WGTKPG" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-WGTKPG');


    $(document).ready(function () {
        $('#enabled-mobile-nav-control').click(function (event) {
            event.stopPropagation();
            event.preventDefault();
            $('body').addClass('left-sidebar-engaged');
        });


        $('.has-country-submenu').click(function (event) {
            $('body').addClass('country-sidebar-engaged');
        });

        $('.sidebar-back-arrow, .sidebar-back-arrow-right, .sidebar-overlay').click(f